# Baseline

W przypadku tego modelu tekst jest przetworzony przez TF-IDF z uni- i bigramami. Następnie na podstawie tych danych trenowany jest model Linear SVM.

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

SEED = 42

df_train = pd.read_csv("../data/df_train.csv", index_col="id")
df_val = pd.read_csv("../data/df_val.csv", index_col="id")
df_test = pd.read_csv("../data/df_test.csv", index_col="id")

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

train_vectorized = vectorizer.fit_transform(df_train["comment_text"])
val_vectorized = vectorizer.transform(df_val["comment_text"])
test_vectorized = vectorizer.transform(df_test["comment_text"])

In [ ]:
svc = LinearSVC(random_state=SEED)

train_labels = np.char.add(df_train["toxic"].astype("string"), df_train["severe_toxic"].astype("string"))
val_labels = np.char.add(df_val["toxic"].astype("string"), df_val["severe_toxic"].astype("string"))

svc.fit(X=train_vectorized, y=train_labels)
val_labels_prd = svc.predict(val_vectorized)

labels = ["00", "01", "10", "11"]
label_names = ["not toxic, not severe toxic", "not toxic, severe toxic", "toxic, not severe toxic", "toxic, severe toxic"]
conf = confusion_matrix(val_labels, val_labels_prd, labels=labels)
conf_frame = pd.DataFrame(conf, columns=np.char.add("(P) ", label_names), index=np.char.add("(T) ", label_names))

conf_frame

# TODO: policzyć na tym accuracy, macro-f1 i roc-auc

,"(P) not toxic, not severe toxic","(P) not toxic, severe toxic","(P) toxic, not severe toxic","(P) toxic, severe toxic"
"(T) not toxic, not severe toxic",1806,0,7,0
"(T) not toxic, severe toxic",0,0,0,0
"(T) toxic, not severe toxic",79,0,77,2
"(T) toxic, severe toxic",6,0,22,1
